<a href="https://colab.research.google.com/github/aneeshbal/SimpylCellCounter/blob/master/examples/scc_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize Session

##### 1. First step, go to "Runtime" -> "Change runtime type", under "Hardware Accelerator, select "GPU"

In [0]:
# please note that tensorflow version 1.0 is selected since original model was created in 1.0

%tensorflow_version 1.x
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import pandas as pd
import math
import os
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file

##### 2. Connect your google drive to the colab session

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


##### 3. Read in functions and load model

In [3]:
%cd /content/drive/My Drive
!git clone https://github.com/aneeshbal/SimpylCellCounter.git
%cd /content/drive/My Drive/SimpylCellCounter/scc/1.1x

/content/drive/My Drive
fatal: destination path 'SimpylCellCounter' already exists and is not an empty directory.
/content/drive/My Drive/SimpylCellCounter/scc/1.1x


In [4]:
from scc import scc
model = load_model('model_save.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Upload Data



##### 4. Under the main "My Drive" folder, upload a folder with your images...wait a few minutes since sometimes the sync can be slow

## Run SCC

##### 5. For the "image_path" variable, type in the appropriate folder name that contains your images

In [0]:
image_path = '/content/drive/My Drive/images' # replace 'sample_images' with appropriate folder name

In [6]:
roots = []
[roots.append(root) for root, dirs, files in os.walk(image_path)]
roots.pop(0)
roots 
## ensure that the printed output looks correct in terms of where the images are located within the images folder

['/content/drive/My Drive/images/D',
 '/content/drive/My Drive/images/B',
 '/content/drive/My Drive/images/Q',
 '/content/drive/My Drive/images/G']

In [33]:
## rememeber, scc parameters are as follows:
## scc(read, model, set_th_value=150, radius=10, circularity_parameter=0.77)
## 'read' = image to be read, don't change this!
## 'model' = CNN model to-be-used, don't change this!
## 'set_th_value' = binary threshold value set at default to 150, can be changed to any value!
## 'radius' = radius for cells set at default to 10, can be changed to any value!
## 'circularity_parameter' = how circular cells should be set at default to 0.77, can be changed to any value! The lower the value, ...
## the less circular cells will be tolerated


output = []
img_track = []
for p in tqdm(roots):
  images = os.listdir(p)
  for im in images:
    img_path = os.path.join(p, im)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    output.append(scc(read=img, model=model, set_th_value=115, circularity_parameter=0.7))
    img_track.append(im)

## Save Output

In [0]:
from time import gmtime, strftime
save_output = pd.DataFrame({'image':img_track,'counts':output})
save_output.set_index('image')
save_output.to_csv('/content/drive/My Drive/scc_output{}.csv'.format(strftime("%Y-%m-%d %H:%M:%S", gmtime())))